In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path
file_path = Path.cwd()
sys.path.append(str(file_path.parents[0] / "Lib3.0" / "lib3.0"))
sys.path.append(str(file_path.parents[0] / "TwoZoneCalculations"))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tkinter import Tk
from tkinter.filedialog import askdirectory
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

from RunSpeedline import Speedline

### Step 1: Select Speedline Directory
Running the following cell will prompt you to select a directory. This directory should contain all of the simulations run for a given speedline. The directory should also contain subdirectories with simulation reports, inlet flow fields, and outlet flow fields.

In [ ]:
# root = Tk()
# root.withdraw()
# root.wm_attributes('-topmost', 1)

speedline_path = file_path

reports_path = speedline_path / 'Reports'
inlet_flow_field_path = speedline_path / "Inlet Flow Fields"
outlet_flow_field_path = speedline_path / "Outlet Flow Fields"
monitor_path = speedline_path / "Monitors"

# root.destroy()

### Step 2: Combine Reports
The following cell reads all of the files in the "Reports" subdirectory and combines the data into a single dataframe.

In [ ]:
df = pd.DataFrame([])
for i, report in enumerate(reports_path.glob("*.txt")):
    if (report.stem == 'Combined'):
        continue
    x = pd.read_table(report, header=None).T
    x.columns = x.iloc[0]
    x = x.drop(x.index[0])
    x = x.drop(x.index[-1])
    if i == 0:        
        df = x
    else:
        df = pd.concat([df, x.astype('float')])
        # print(x.loc[1])

df.to_csv(reports_path / 'Combined.csv', index=False)

### Step 3: Determine Convergence
The following code will generate plots that show how well each simulation converged. If you determine that a simulation is converged, move on. If a simulation is not converged, note the operating conditions, the simulation path, and briefly explain why you believe it is not converged in the progress spreadsheet.

In [ ]:
n_iterations = 2000
monitors = {}
for sim in monitor_path.iterdir():
    # print(sim)
    if (sim.is_dir()) and ("@" not in sim.name):
        df = pd.DataFrame([])
        for monitor in sim.glob("*.csv"):
            if (monitor.stem not in ['Physical Time']):
                load = pd.read_csv(monitor, usecols=[1])
                df = pd.concat([df, load[-n_iterations:]], axis=1)
        df.columns = [header.split(':')[0] for header in df.columns]
        df = df.reset_index(drop=True)
        monitors[sim.name] = df

bigdf = pd.concat(monitors, keys=monitors.keys())

In [ ]:
all_sims = list(monitors.keys())

def plot_monitor(monitor, case):
    df = monitors[case]
    curve = hv.Curve(df,"Iteration", monitor)
    curve.opts(height=450,
               width=950, 
               framewise=True)
    return curve

def plot_speedline(yAxis):
    scatter = hv.Scatter(bigdf[yAxis].groupby(level=0).last().reset_index())
    scatter.opts(height=500, 
                 width=750, 
                 framewise=True, 
                 color='black', 
                 marker='circle', 
                 size=10, 
                 xlabel="Mass Flow Rate")
    return scatter

hv.output(widget_location='top_left', rows=1, cols=2)
monitor_filter = hv.Dimension("Monitor", values=df.columns)
sim_filter = hv.Dimension("Simulations", values=all_sims)

dmap1 = hv.DynamicMap(plot_speedline, kdims=[monitor_filter])
dmap2 = hv.DynamicMap(plot_monitor, kdims=[monitor_filter, sim_filter])
hv.Layout([dmap1, dmap2]).cols(1)

### Step 4: Calculate the Two-Zone Parameters
The following code reads all of the inlet/outlet files and calculates the desired parameters. An output file titled "twoZoneAnalysis.csv" is saved to the speedline directory.

#### Required Inputs:
These inputs can be found in the master spreadsheet, COMPAL/PUMPAL, AxCent, or the simulation files.



- Period (The number of impeller main blades)
- Beta (The exit blade angle in degrees)
- N (The impeller rotation rate)
- Tt0 (Upstream total temperature in K)
- Pt0 (Upstream total pressure in Pa)

In [ ]:
Period = 7
Beta = -25
N = 120000

Tt0 = 298.15
Pt0 = 101325

speedline = Speedline(Beta, N, (Tt0, Pt0), Period, cutoff_var="Cm/Cm,i", cutoff=1, interpolate=True)
speedline.run(inlet_flow_field_path, outlet_flow_field_path)
speedline.output.to_csv(speedline_path / "twoZoneAnalysis.csv", index=False)